In [58]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load Dataset

In [24]:
path = '..\Data_Source\Social_Network_Ads.csv'
df = pd.read_csv(path)

# Data Inspection

In [25]:
df.shape

(400, 5)

In [26]:
df.dtypes

User ID             int64
Gender             object
Age                 int64
EstimatedSalary     int64
Purchased           int64
dtype: object

In [27]:
df.isna().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [28]:
df.Gender.unique()

array(['Male', 'Female'], dtype=object)

In [29]:
df.Purchased.unique()

array([0, 1], dtype=int64)

In [30]:
df.describe()

,User ID,Age,EstimatedSalary,Purchased
count,4.000000e+02,400.000000,400.000000,400.000000
mean,1.569154e+07,37.655000,69742.500000,0.357500
std,7.165832e+04,10.482877,34096.960282,0.479864
min,1.556669e+07,18.000000,15000.000000,0.000000
25%,1.562676e+07,29.750000,43000.000000,0.000000
50%,1.569434e+07,37.000000,70000.000000,0.000000
75%,1.575036e+07,46.000000,88000.000000,1.000000
max,1.581524e+07,60.000000,150000.000000,1.000000


# Feature Encoding

In [31]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

# Feature Selection

In [32]:
df.columns

Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')

In [33]:
X = df.drop(['User ID', 'Purchased'], axis=1)
y = df.Purchased

# Train Test Split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature Scaling

In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building Base Models

In [59]:
base_models = [
    ('lr', LogisticRegression()),
    ('dt', DecisionTreeClassifier(max_depth=3)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('svm', SVC(kernel='rbf', C=1, gamma='scale'))
]

In [60]:
meta_model = LogisticRegression()

# Building Stacking Classifier

In [61]:
classifier = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

classifier.fit(X_train_scaled, y_train)

,estimators,"[('lr', ...), ('dt', ...), ...]"
,final_estimator,LogisticRegression()
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


# Model Evaluation

In [62]:
y_pred = classifier.predict(X_test_scaled)

In [63]:
# Accuracy
accuracy_score(y_test, y_pred)

0.9333333333333333